In [16]:
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns
import os

# =============================
# CONFIG
# =============================
DB_USER = "postgres"
DB_PASSWORD = "kakekjelek1"   # <-- CHANGE
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "job_market_intelligence"

DATA_DIR = "../data"
ASSET_DIR = "../assets"

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(ASSET_DIR, exist_ok=True)

engine = create_engine(
    f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

sns.set_theme(style="whitegrid")

# =============================
# QUERY 1 — Salary Trends by State
# =============================
query1 = """
SELECT
    l.state,
    COUNT(*) AS job_count,
    ROUND(AVG(f.salary_min), 0) AS avg_salary_min,
    ROUND(AVG(f.salary_max), 0) AS avg_salary_max,
    ROUND(AVG((f.salary_min + f.salary_max) / 2), 0) AS avg_salary_midpoint,
    MIN(f.salary_min) AS lowest_salary_min,
    MAX(f.salary_max) AS highest_salary_max
FROM fact_job_postings f
JOIN dim_location l ON f.location_id = l.location_id
WHERE f.salary_min IS NOT NULL
  AND f.salary_max IS NOT NULL
GROUP BY l.state
HAVING COUNT(*) >= 5
ORDER BY avg_salary_midpoint DESC;
"""

df_q1 = pd.read_sql(query1, engine)
df_q1.to_csv(f"{DATA_DIR}/query1_salary_by_state.csv", index=False)

plt.figure(figsize=(12, 6))
sns.barplot(data=df_q1, x="state", y="avg_salary_midpoint")
plt.xticks(rotation=45)
plt.title("Average Salary Midpoint by State")
plt.tight_layout()
plt.savefig(f"{ASSET_DIR}/q1_salary_by_state.png")
plt.close()


# =============================
# QUERY 2 — Salary Trends by Seniority Level
# =============================
query2 = """
SELECT
    s.seniority_level,
    COUNT(*) AS job_count,
    ROUND(AVG(f.salary_min), 0) AS avg_salary_min,
    ROUND(AVG(f.salary_max), 0) AS avg_salary_max,
    ROUND(AVG((f.salary_min + f.salary_max) / 2), 0) AS avg_salary_midpoint,
    MIN(f.salary_min) AS lowest_salary_min,
    MAX(f.salary_max) AS highest_salary_max
FROM fact_job_postings f
JOIN dim_seniority s ON f.seniority_id = s.seniority_id
WHERE f.salary_min IS NOT NULL
  AND f.salary_max IS NOT NULL
GROUP BY s.seniority_level
ORDER BY avg_salary_midpoint DESC;
"""

df_q2 = pd.read_sql(query2, engine)
df_q2.to_csv(f"{DATA_DIR}/query2_salary_by_seniority.csv", index=False)

plt.figure(figsize=(8, 5))
sns.barplot(data=df_q2, x="seniority_level", y="avg_salary_midpoint")
plt.title("Average Salary Midpoint by Seniority Level")
plt.tight_layout()
plt.savefig(f"{ASSET_DIR}/q2_salary_by_seniority.png")
plt.close()


# =============================
# QUERY 3 — Top Companies by Hiring Volume
# =============================
query3 = """
SELECT
    c.company_name,
    COUNT(*) AS job_count,
    ROUND(AVG((f.salary_min + f.salary_max) / 2), 0) AS avg_salary_midpoint
FROM fact_job_postings f
JOIN dim_company c ON f.company_id = c.company_id
WHERE f.salary_min IS NOT NULL
  AND f.salary_max IS NOT NULL
GROUP BY c.company_name
HAVING COUNT(*) >= 5
ORDER BY job_count DESC
LIMIT 15;
"""

df_q3 = pd.read_sql(query3, engine)
df_q3.to_csv(f"{DATA_DIR}/query3_top_companies.csv", index=False)

plt.figure(figsize=(12, 6))
sns.barplot(data=df_q3, y="company_name", x="job_count")
plt.title("Top 15 Companies by Hiring Volume")
plt.tight_layout()
plt.savefig(f"{ASSET_DIR}/q3_top_companies.png")
plt.close()


# =============================
# QUERY 4 — Salary by Company Type
# =============================
query4 = """
SELECT
    c.company_type,
    COUNT(*) AS job_count,
    ROUND(AVG(f.salary_min), 0) AS avg_salary_min,
    ROUND(AVG(f.salary_max), 0) AS avg_salary_max,
    ROUND(AVG((f.salary_min + f.salary_max) / 2), 0) AS avg_salary_midpoint
FROM fact_job_postings f
JOIN dim_company c ON f.company_id = c.company_id
WHERE f.salary_min IS NOT NULL
  AND f.salary_max IS NOT NULL
  AND c.company_type IS NOT NULL
GROUP BY c.company_type
HAVING COUNT(*) >= 5
ORDER BY avg_salary_midpoint DESC;
"""

df_q4 = pd.read_sql(query4, engine)
df_q4.to_csv(f"{DATA_DIR}/query4_salary_by_company_type.csv", index=False)

plt.figure(figsize=(10, 5))
sns.barplot(data=df_q4, x="company_type", y="avg_salary_midpoint")
plt.title("Average Salary Midpoint by Company Type")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(f"{ASSET_DIR}/q4_salary_by_company_type.png")
plt.close()


# =============================
# QUERY 5 — Easy Apply vs Salary
# =============================
query5 = """
SELECT
    f.easy_apply,
    COUNT(*) AS job_count,
    ROUND(AVG((f.salary_min + f.salary_max) / 2), 0) AS avg_salary_midpoint,
    MIN(f.salary_min) AS lowest_salary_min,
    MAX(f.salary_max) AS highest_salary_max
FROM fact_job_postings f
WHERE f.salary_min IS NOT NULL
  AND f.salary_max IS NOT NULL
GROUP BY f.easy_apply
ORDER BY f.easy_apply;
"""

df_q5 = pd.read_sql(query5, engine)
df_q5.to_csv(f"{DATA_DIR}/query5_easy_apply_vs_salary.csv", index=False)

plt.figure(figsize=(6, 5))
sns.barplot(data=df_q5, x="easy_apply", y="avg_salary_midpoint")
plt.title("Easy Apply vs Salary Midpoint")
plt.tight_layout()
plt.savefig(f"{ASSET_DIR}/q5_easy_apply_vs_salary.png")
plt.close()


# =============================
# QUERY 6 — Visualization Helper Dataset
# =============================
query6 = """
SELECT
    l.state,
    COUNT(*) AS job_count,
    ROUND(AVG(f.salary_min), 0) AS avg_salary_min,
    ROUND(AVG(f.salary_max), 0) AS avg_salary_max,
    ROUND(AVG((f.salary_min + f.salary_max) / 2), 0) AS avg_salary_midpoint
FROM fact_job_postings f
JOIN dim_location l ON f.location_id = l.location_id
WHERE f.salary_min IS NOT NULL
  AND f.salary_max IS NOT NULL
GROUP BY l.state;
"""

df_q6 = pd.read_sql(query6, engine)
df_q6.to_csv(f"{DATA_DIR}/query6_bi_salary_dataset.csv", index=False)

plt.figure(figsize=(12, 6))
sns.barplot(data=df_q6, x="state", y="avg_salary_midpoint")
plt.xticks(rotation=45)
plt.title("BI Dataset — Avg Salary by State")
plt.tight_layout()
plt.savefig(f"{ASSET_DIR}/q6_bi_salary_dataset.png")
plt.close()


# =============================
# FINAL LOG
# =============================
print("ALL QUERIES EXECUTED SUCCESSFULLY")
print("CSVs saved to:", DATA_DIR)
print("Charts saved to:", ASSET_DIR)


ALL QUERIES EXECUTED SUCCESSFULLY
CSVs saved to: ../data
Charts saved to: ../assets
